**O código abaixo importa as bibliotecas nescessarias e define a função mine()**

In [0]:
import requests
import tarfile
import datetime
from sys import argv
from time import sleep
from google.colab import files


In [0]:
#A função mine é responsavel por fazer uma requisição a API do Twitter
#e salvar os dados recebidos em um arquivo
def mine(name, label, startDate, endDate, TOKEN, ):
        f = open(name+'.json','wt')
        #Aqui pode ser nescessario mudar de 30day para fullarchive
        response = requests.get('https://api.twitter.com/1.1/tweets/search/30day/'+label+'.json',
            params={'query':'lang:pt place_country:br',
                    'fromDate':startDate,
                    'toDate':endDate
                },
            headers={'Authorization':'bearer {}'.format(TOKEN)})
        f.write(response.text)
        #files.download(name+'.json')
        return response.status_code

Coloque a sua **API_KEY** e **API_SECRET** no código abaixo para receber a acess_token, que sera usada para fazer as requisições

In [3]:
#Esta função retorna o token que sera usado para fazer as requisições
def getToken():
  API_KEY = 'g1b1kskNcopO7S1T1BkHkXIWF'   #'SUA KEY AQUI'
  API_SECRET = 'FDviS5EFqSbvzfBKazvPDnRPYs4t2jpiRwdQfcX6wEQIExdlgP'   #'SUA SECRET KEY AQUI'

  response = requests.post('https://api.twitter.com/oauth2/token',
                          auth=(API_KEY,API_SECRET),
                          params={'grant_type':'client_credentials'})
  return response.json()["access_token"]

print(getToken())

AAAAAAAAAAAAAAAAAAAAAP8v%2FQAAAAAAWU61fPOObPcrLSPTZCvounTlirE%3DV8w6wmGQnepTu5YTVGdWRljmoGmf680MBGZyPMSzC8dpaLW45P


Neste bloco definimos a data de inicio e a função parseTime, que transforma o formato das datas.

Ex:

```
>>>data_inicio
2019-08-01 12:00:00 //Permite operações de tempo
>>>parseTime(data_incio)
20190801120000  //Formato aceito pela API
```







In [41]:
  ano = 2019#ano
  mes = 10#mes
  dia = 3#dia
  hora = 12#hora
  minuto = 00#minutos
  data_inicio = datetime.datetime(ano,mes,dia,hora,minuto)
  def parseTime(data_inicio):
    return data_inicio.isoformat().translate({ord('-'):None, ord(':'):None, ord('T'):None})[0:-2]
  print(data_inicio)
  print(parseTime(data_inicio))

2019-10-03 12:00:00
201910031200


Aqui é onde é feita a requisição, 

In [43]:
TOKEN = getToken()
name = 'arq'
label = 'Sample30'
startDate = parseTime(data_inicio) #AAAAMMDDhhmm

#Cria o arquivo tar que vai conter todos os tweets
tf = tarfile.open("tweets.tar","w")

#O for abaixo faz uma requisição a cada iteração, e aumenta 
#a hora do startDate em 1 hora. Mude o valor no range e defina
#um startDate valido.
for aux in range(2):
    endDate = parseTime(data_inicio + datetime.timedelta(hours=1+aux))
    ret = mine(name + str(aux), label, startDate, endDate, TOKEN)
    tf.add(name+str(aux)+".json")
    startDate = endDate
    print(ret)
    sleep(1)

tf.close()

201910031200
201910031300
200
201910031300
201910031400
200


In [0]:
TOKEN = getToken()
name = 'agosto12'
label = 'UselessLabel'
startDate = data_inicio#'AAAAMMDDHHmm'

#Cria o arquivo tar que vai conter todos os tweets
tf = tarfile.open("tweets.tar","w")

#O for abaixo faz uma requisição a cada iteração, e aumenta 
#o dia do startDate em 1 dia. Mude o valor no range e defina
#um startDate valido.
for aux in range(30):
    endDate = parseTime(data_inicio + datetime.deltatime(days=1+aux))
    ret = mine(name + str(aux), label, startDate, endDate, TOKEN)
    tf.add(name+str(aux)+".json")
    startDate = endDate
    print(ret)
    sleep(1)

tf.close()

